In [ ]:
!pip install mlflow
!pip install findspark
!pip install pyspark

In [ ]:
import findspark
import pyspark
findspark.init()

from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.option("header","true").option("inferSchema","true").csv("./models/data/news_category_test.csv").toDF("label","text")
df.show()

In [ ]:
import mlflow

loaded_model_udf = mlflow.pyfunc.spark_udf(spark, 
                                           model_uri="runs:/478eacd5b1f04010be3b633b7e4434f4/classification", 
                                           result_type='string')

In [ ]:
test = df.select(df.text, df.label, loaded_model_udf(df.text).alias("prediction"))
test.show()

7600


2024/07/30 12:33:46 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.12`, differs from the version of Python that is currently running, `Python 3.11.5`, and may be incompatible
2024/07/30 12:33:47 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/adityakulkarni/UTD

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/adityakulkarni/UTD/6350_BDA/media-bias-detector/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/adityakulkarni/UTD/6350_BDA/media-bias-detector/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/adityakulkarni/UTD/6350_BDA/media-bias-detector/.venv/lib

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/adityakulkarni/UTD/6350_BDA/media-bias-detector/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/adityakulkarni/UTD/6350_BDA/media-bias-detector/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/adityakulkarni/UTD/6350_BDA/media-bias-detector/.venv/lib

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

RuntimeError: Failed to import transformers.models.auto.processing_auto because of the following error (look up to see its traceback):
numpy.core.umath failed to import